In [4]:
!pip install category_encoders

     |████████████████████████████████| 81kB 5.2MB/s 


# Sources


In [ ]:
#sources:
#https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter
#https://www.kaggle.com/namanj27/new-baseline-pytorch-moa
#https://www.kaggle.com/headsortails/explorations-of-action-moa-eda

## Import libraries

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [6]:
import os
import gc
import random
import math
import time
from sklearn.feature_selection import VarianceThreshold as V
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import log_loss

import category_encoders as ce

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Read The data

In [7]:
df_train = pd.read_csv('drive/MyDrive/lish-moa/train_features.csv')
df_test = pd.read_csv('drive/MyDrive/lish-moa/test_features.csv')
df_ss = pd.read_csv('drive/MyDrive/lish-moa/sample_submission.csv')
df_ts = pd.read_csv('drive/MyDrive/lish-moa/train_targets_scored.csv')
df_tns = pd.read_csv('drive/MyDrive/lish-moa/train_targets_nonscored.csv')

In [ ]:
print(f'shapes:\n train: {df_train.shape} \n target:{df_ts.shape} \n test: {df_test.shape}')

shapes:
 train: (23814, 876) 
 target:(23814, 207) 
 test: (3982, 876)


# Task
 <span style="color:#000000; font-family: 'Georgia'; font-size: 1.2em;">
Our task is multilabel classification. We are given 876 features(including id's of patients) and target consists of 206 features(each feature takes 0 or 1). <br>
df_train is our training set and df_ss is the target dataframe. Our task is to get target values for df_test. <br>
we additionally given df_tns - train_targets_nonscored. It is an additional (optional) binary responses for the training data. These are not predicted nor scored. I have no idea, why they gave it, so i didn't use it at all(as majority of competitors) <br>
link to the competition: https://www.kaggle.com/c/lish-moa/data
</span>

In [15]:
df_train.head(2)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.183,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.152,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371


In [13]:
df_ts.head(2)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df_test.head(2)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,-0.214,0.2221,-0.3260,1.9390,-0.2305,-0.3670,1.304,1.4610,0.0043,0.6816,-0.2304,-0.0635,-0.2030,-0.6821,-0.6242,0.1297,-0.0338,0.3372,0.2254,0.4795,0.7642,0.6638,-0.2480,-0.1183,-0.4847,-0.0179,-0.8204,-0.5296,-1.5070,-0.0144,...,-0.1353,0.0494,0.8939,0.227,0.2876,-0.3065,0.6519,-0.8156,-1.4960,0.3796,0.0877,-1.0230,-0.0206,-0.4149,-0.6258,-0.2688,0.4403,-0.4900,0.2910,0.0473,-0.0914,0.3087,-0.0612,-0.9128,-0.9399,0.0173,0.0519,-0.0035,-0.5184,-0.3485,0.0981,0.7978,-0.143,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.151,-0.775
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,-1.882,0.4022,-0.3528,0.1271,0.9303,0.3173,-1.012,-0.3213,0.0607,-0.5389,-0.8030,-1.0600,-0.0978,-0.8156,-0.6514,0.6812,0.5246,0.0000,0.5030,-0.1500,-0.1433,2.0910,-0.6556,-0.6012,-0.4104,-0.0580,-0.3608,0.2197,-0.7101,1.3430,...,-0.7458,0.0458,-0.3644,-1.818,-0.0358,-0.7925,-0.2693,-0.0938,-0.1833,-0.7402,-1.4090,0.1987,0.0460,-1.3520,-0.3445,-0.0909,-0.6337,-0.5788,-0.7885,0.0996,-1.9480,-1.2720,-0.7223,-0.5838,-1.3620,-0.7671,0.4881,0.5913,-0.4333,0.1234,-0.1190,-0.1852,-1.031,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.381,-0.730


# QuanTile Transformation
 <span style="color:#000000; font-family: 'Georgia'; font-size: 1.2em;"> quatile transformation is one of the preprocssing steps, which makes the features to follow normal distribution. It also helps to reduce the effect of outliers</span>

In [ ]:
from sklearn.preprocessing import QuantileTransformer
num_cols = df_train.drop(columns = ['cp_time']).select_dtypes([np.number]).columns
qt = QuantileTransformer(n_quantiles=100, random_state=42, output_distribution = 'normal')

def quantileTransf(num_cols, df):
    for col in num_cols:
        qt.fit(df[col][:,None])
        df[col] = qt.transform(df[col][:,None])
    return df 

df_train = quantileTransf(num_cols, df_train)
df_test = quantileTransf(num_cols, df_test)    

# Drop ctrl_vehicle
 <span style="color:#000000; font-family: 'Georgia'; font-size: 1.2em;"> We drop the rows which has cp_type of ctl_vehicle, since their target values always 0, so we don't need them in the training, just in the end fill with 0 the rows with ctl_vehicle </span>

In [ ]:
def drop_ctlVecile(df):
    df = df[df['cp_type']!='ctl_vehicle'].reset_index(drop=True)
    return df

df_train = drop_ctlVecile(df_train)    
test = drop_ctlVecile(df_test)

data_with = pd.merge(df_train, df_ts, on='sig_id')
data = data_with[data_with['cp_type'] != 'ctl_vehicle']
print(f'New shapes: \n data(train(no ctl_vehicle) + target): {data.shape} \n test(no ctl_vehicle):                 {df_test.shape}')

New shapes: 
 data(train(no ctl_vehicle) + target): (21948, 1082) 
 test(no ctl_vehicle):                 (3982, 876)


# PCA - dimensionality reduction
 <span style="color:#000000; font-family: 'Georgia'; font-size: 1.2em;"> One might say that we don't need PCA, when you use Neural Network, since NN itself is good dimensionality reduction method. However, here the we concatenate values after PCA to the original values since it somehow improves the accuracy. This was noticed in one of the sources, i mentioned above </span>

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

gene = data.loc[:,'g-0': 'g-771']
gene_test = test.loc[:,'g-0': 'g-771']

scaler = StandardScaler()
scaled_gene = scaler.fit_transform(gene)
scaled_test_gene = scaler.transform(gene_test)
pca = PCA(0.95)
pca_data = pca.fit_transform(scaled_gene)
pca_test = pca.transform(scaled_test_gene)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

cell = data.loc[:,'c-0': 'c-99']
cell_test = test.loc[:,'c-0': 'c-99']

#--------------------------------------------------------------------------
scaler = StandardScaler()
scaled_cell = scaler.fit_transform(cell)
scaled_test_cell = scaler.transform(cell_test)

pca2 = PCA(0.95)
pca_data2 = pca2.fit_transform(scaled_cell)
pca_test2 = pca2.transform(scaled_test_cell)

In [ ]:
loadings = pca_data
labels = ['PC' + str(x) + '_g' for x in range(1, pca_data.shape[1] + 1)]
pca_gene = pd.DataFrame(loadings, columns = labels)
loadings2 = pca_data2
labels2 = ['PC' + str(x) + '_c' for x in range(1, pca_data2.shape[1] + 1)]
pca_cell = pd.DataFrame(loadings2, columns = labels2)
frames = [pca_gene, pca_cell]
df_gene_cell = pd.concat(frames, axis = 1)
print(f'So, the size of the new dataframe, which is concatenation of gene and cell features after PCA(0.95) is {df_gene_cell.shape}')

So, the size of the new dataframe, which is concatenation of gene and cell features after PCA(0.95) is (21948, 685)


In [ ]:
loadings = pca_test
labels = ['PC' + str(x) + '_g' for x in range(1, pca_test.shape[1] + 1)]
pca_test_gene = pd.DataFrame(loadings, columns = labels)
loadings2 = pca_test2
labels2 = ['PC' + str(x) + '_c' for x in range(1, pca_test2.shape[1] + 1)]
pca_test_cell = pd.DataFrame(loadings2, columns = labels2)
frames = [pca_test_gene, pca_test_cell]
df_pca_test = pd.concat(frames, axis = 1)
print(f'So, the size of the new dataframe, which is concatenation of gene and cell features after PCA(0.95) is {df_pca_test.shape}')

So, the size of the new dataframe, which is concatenation of gene and cell features after PCA(0.95) is (3624, 685)


In [ ]:
data = data.merge(df_gene_cell, left_index=True,right_index=True, how='left')
test = test.merge(df_pca_test, left_index=True,right_index=True, how='left')


# Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class base(nn.Module):
    def __init__(self):
        super(base, self).__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d(1560),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(1560, 1024)),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.utils.weight_norm(nn.Linear(1024, 1024)),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.utils.weight_norm(nn.Linear(1024, 206)),
            
            
        
        )
       
    def forward(self, cont_x):
        x = self.model(cont_x)
        return x

## Train and test wrappers
 <span style="color:#000000; font-family: 'Georgia'; font-size: 1.2em;"> We have to wrap our data in order to be able to use them in DataLoader</span>

In [ ]:
class TrainDataset(Dataset):
    def __init__(self,df, numeric,  labels):
        self.numeric = df[numeric].values
        self.labels = labels
    def __len__(self):
        return len(self.numeric)
    def __getitem__(self, idx):
        numerical_tensor = torch.FloatTensor(self.numeric[idx])
        label = torch.tensor(self.labels[idx]).float()
        
        return numerical_tensor, label

    
class TestDataset(Dataset):
    def __init__(self,df, numeric):
        self.numeric = df[numeric].values
    def __len__(self):
        return len(self.numeric)
    def __getitem__(self, idx):
        numerical_tensor = torch.FloatTensor(self.numeric[idx])
        
        return numerical_tensor

    

# Variance Threshold
 <span style="color:#000000; font-family: 'Georgia'; font-size: 1.2em;"> I thought of this as another preprocessing method, but it greatly reduced the accuracy</span>


In [ ]:
'''
from sklearn.feature_selection import VarianceThreshold as V
selector = V(0.8)
my_cols = [i for i in test.columns if i not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
selected_data = selector.fit_transform(data.loc[:,my_cols])
selected_test = selector.transform(test.loc[:,my_cols])

In [ ]:
'''
labels = ['s' + str(x)  for x in range(1, selected_data.shape[1] + 1)]
selected_data = pd.DataFrame(selected_data, columns = labels)
selected_test = pd.DataFrame(selected_test, columns = labels)

target_cols = [i for i in df_ts.columns if i != 'sig_id']
categorical = ['cp_time', 'cp_dose']
target_cols += categorical
data = selected_data.merge(data[target_cols], left_index=True, right_index=True, how='left')
test = selected_test.merge(data[categorical], left_index=True, right_index=True, how='left')

# Categorical Variables
 <span style="color:#000000; font-family: 'Georgia'; font-size: 1.2em;">
we have some categorical values like cp_time, cp_dose, so i get their dummy values. I actually tried training without these categorical variables, but using their dummy variables improved the result </span>

In [ ]:
cp_time = pd.get_dummies(data['cp_time'], drop_first=True)
cp_dose = pd.get_dummies(data['cp_dose'], drop_first=True)
cp_time2 = pd.get_dummies(test['cp_time'], drop_first=True)
cp_dose2 = pd.get_dummies(test['cp_dose'], drop_first=True) 
test = test.merge(cp_time2, left_index=True,right_index=True, how='left')
test = test.merge(cp_dose2, left_index=True,right_index=True, how='left')
data = data.merge(cp_time, left_index=True,right_index=True, how='left')
data = data.merge(cp_dose, left_index=True,right_index=True, how='left')


numerical =  data.select_dtypes([np.number])
categorical = ['cp_time', 'cp_dose']
target_cols = df_ts.drop(columns = ['sig_id']).columns 
numerical = [i for i in numerical.columns if i not in categorical]
numerical = [i for i in numerical if i not in target_cols]
data['cp_time'] = data['cp_time'].map({24: 0, 48: 1, 72: 2}) 
data['cp_dose'] = data['cp_dose'].map({'D1': 3, 'D2': 4}) 
test['cp_time'] = test['cp_time'].map({24: 0, 48: 1, 72: 2}) 
test['cp_dose'] = test['cp_dose'].map({'D1': 3, 'D2': 4})

This ones simple training, validation and inference(for test set) functions \\
source: https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def train(model, train_loader, scheduler, optimizer, device):
    losses = AverageMeter()

    model.train()
    for step, (numer, y) in enumerate(train_loader):
        
        numer, y = numer.to(device), y.to(device)
        batch_size = numer.size(0)

        pred = model(numer)
        loss = nn.BCEWithLogitsLoss()(pred, y)
        losses.update(loss.item(), batch_size)
       
    #    val_preds.append(pred.sigmoid().detach().cpu().numpy())

        
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1000)

        scheduler.step()
        optimizer.step()
        optimizer.zero_grad()
        
    return losses.avg
        

def validation(model, valid_loader, device):
    losses = AverageMeter()
    model.eval()
    val_preds = []
    for step, (numer,  y) in enumerate(valid_loader):
        numer,  y = numer.to(device),  y.to(device)
        batch_size = numer.size(0)
        with torch.no_grad():
            pred = model(numer)
        loss = nn.BCEWithLogitsLoss()(pred, y)
        losses.update(loss.item(), batch_size)
        val_preds.append(pred.sigmoid().detach().cpu().numpy())
    val_preds = np.concatenate(val_preds)
    
    return losses.avg,val_preds        


def inference_fn(test_loader, model, device):

    model.eval()
    preds = []

    for step, (cont_x) in enumerate(test_loader):

        cont_x = cont_x.to(device)

        with torch.no_grad():
            pred = model(cont_x)

        preds.append(pred.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds)

    return preds
       
            
   

# Loop
here we use multilableStratifiedKfold to divide our data into training and validation. This is the same as StratifiedKfold, but since our task is multilabel classification, we use multilabelStratifiedKFold

In [ ]:
#!pip install iterative-stratification

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
kf =  MultilabelStratifiedKFold(n_splits=5, random_state=42, shuffle=True)

kf.get_n_splits(data) 


train_cols = data.loc[:,"cp_time": ].columns
target_cols = df_ts.loc[:,"5-alpha_reductase_inhibitor": ].columns

def run_different_seed(data, target_cols, numerical, categorical, seed = 42):
    oof = np.zeros((len(data), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))

    seed_everything(seed = seed)
    
    print(f'Seed: {seed}')
    print('--------------------------------------')
    for fold_num, (train_index, test_index) in enumerate(kf.split(data, data[target_cols])):
        print(f'fold_num: {fold_num}')

        X_train, X_test = data.loc[:, train_cols], data.loc[:, train_cols]
        X_train, X_test = X_train.iloc[train_index], X_test.iloc[test_index]

        y_train, y_test = data[target_cols], data[target_cols]
        y_train, y_test = y_train.iloc[train_index].values, y_test.iloc[test_index].values

        train_dataset = TrainDataset(X_train, numerical, y_train)
        validation_dataset = TrainDataset(X_test, numerical,y_test)

        train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True, 
                                  num_workers=4, pin_memory=True, drop_last=True)
        valid_loader = DataLoader(validation_dataset, batch_size = 32, shuffle=False, 
                                  num_workers=4, pin_memory=True, drop_last=False)

        model = base()
        model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay = 1e-6)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                                  max_lr=1e-2, epochs = 10, steps_per_epoch=len(train_loader))
        best_loss = np.inf
        for epochs in range(5):

            train_loss = train(model,train_loader, scheduler, optimizer, device)
            print(f'epoch: {epochs}')
            print(f'train: {train_loss}')
            valid_loss, val_preds = validation(model,valid_loader, device)
            print(f'validation: {valid_loss}')

            if valid_loss < best_loss:
                best_loss = valid_loss
                _oof = np.zeros((len(data), len(target_cols)))
                _oof[test_index] = val_preds
                print(f'saving the best model')
                torch.save(model.state_dict(), f"fold{fold_num}_seed{seed}.pth")

        test_dataset = TestDataset(test, numerical)#, categorical)

        test_loader = DataLoader(test_dataset, batch_size = 32, shuffle=False, 
                                  num_workers=4, pin_memory=True)
        model.load_state_dict(torch.load(f"fold{fold_num}_seed{seed}.pth"))
        model.to(device)
        print('------------------------------------------------------------------------')
        _predictions = inference_fn(test_loader, model, device)
        torch.cuda.empty_cache()
        
        oof += _oof
        predictions += _predictions / 5
        
    score = 0
    for i in range(data[target_cols].values.shape[1]):
        _score = log_loss(data[target_cols].values[:,i], oof[:,i])
        score += _score / data[target_cols].values.shape[1]
    print(f"CV score: {score}")
    print('-------------------------------------------------------------') 
    return oof, predictions

we use different seed for neural networks, it allows us to explore more weight spaces

In [ ]:
seeds = [1, 2, 3, 4, 5]
oof = np.zeros((len(data), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))
for s in seeds:
    _oof, _predictions = run_different_seed(data,target_cols, numerical, categorical, seed = s)
    oof += _oof / len(seeds)
    predictions += _predictions / len(seeds)
score = 0
for i in range(data[target_cols].values.shape[1]):
    _score = log_loss(data[target_cols].values[:,i], oof[:,i])
    score += _score / data[target_cols].values.shape[1]
print(f"average CV score: {score}")

Seed: 1
--------------------------------------
fold_num: 0
epoch: 0
train: 0.20798605027690148
validation: 0.025065140463496668
saving the best model
epoch: 1
train: 0.020395829367798067
validation: 0.018458024897784468
saving the best model
epoch: 2
train: 0.01885358028344973
validation: 0.01785461714020283
saving the best model
epoch: 3
train: 0.017992535949324387
validation: 0.017570902978267376
saving the best model
epoch: 4
train: 0.017311564254078225
validation: 0.017312067488297514
saving the best model
------------------------------------------------------------------------
fold_num: 1
epoch: 0
train: 0.20848177641929283
validation: 0.021311235508472068
saving the best model
epoch: 1
train: 0.02078346256336646
validation: 0.0185871114916166
saving the best model
epoch: 2
train: 0.01894108926789006
validation: 0.01812619383792671
saving the best model
epoch: 3
train: 0.01807070003795254
validation: 0.01780620363364866
saving the best model
epoch: 4
train: 0.017407719497942793
va

In [ ]:
test[target_cols] = predictions
cols = [i for i in df_ts.columns]
cols = [i for i in df_ts.columns if i != 'sig_id']
sub = df_ss.drop(columns=[i for i in cols]).merge(test[cols], left_index=True,right_index=True, how='left').fillna(0)
sub.to_csv('submission.csv', index=False)
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000656,0.001110,0.001501,0.011179,0.018964,0.003942,0.001988,0.006558,0.000127,0.011284,0.017505,0.000992,0.000207,0.000646,0.000768,0.000564,0.002091,0.004393,0.006480,0.001900,0.002178,0.004815,0.000328,0.001362,0.000311,0.000198,0.000712,0.000699,0.004102,0.001399,0.001335,0.003091,0.003294,0.000113,0.000067,0.000258,0.002213,0.000196,0.000471,...,0.003062,0.000480,0.003793,0.000542,0.000539,0.004086,0.000263,0.000838,0.000954,0.001368,0.012772,0.012890,0.002398,0.003114,0.001192,0.001090,0.017388,0.002069,0.000770,0.000302,0.000312,0.002613,0.000098,0.001011,0.001319,0.001151,0.000497,0.002148,0.000818,0.000658,0.000584,0.000896,0.003382,0.001208,0.001031,0.000390,0.002652,0.001277,0.002801,0.001354
1,id_001897cda,0.000107,0.000556,0.001100,0.002981,0.004086,0.001717,0.002248,0.006468,0.003138,0.009283,0.010339,0.002438,0.000138,0.006328,0.000197,0.000217,0.000887,0.001875,0.002566,0.002664,0.002142,0.001725,0.000141,0.001428,0.000517,0.001021,0.000416,0.000871,0.002008,0.000798,0.000260,0.002411,0.001269,0.001467,0.000056,0.000441,0.004638,0.002814,0.008467,...,0.003099,0.001215,0.000749,0.000052,0.001433,0.003014,0.000416,0.005070,0.000929,0.002051,0.008173,0.006312,0.000714,0.000648,0.001394,0.002070,0.007256,0.000362,0.018279,0.000089,0.001888,0.002605,0.000289,0.000490,0.000324,0.000707,0.000328,0.001810,0.006695,0.001238,0.000204,0.000941,0.001852,0.000172,0.012968,0.000216,0.014801,0.000645,0.002079,0.001103
2,id_002429b5b,0.000227,0.000201,0.001085,0.008201,0.011683,0.002613,0.001780,0.002980,0.000041,0.005783,0.014229,0.001270,0.000091,0.002277,0.000471,0.000627,0.001293,0.003273,0.001689,0.001173,0.001005,0.001787,0.000151,0.001815,0.000306,0.000345,0.000360,0.000609,0.002872,0.001310,0.000467,0.001351,0.001695,0.000223,0.000055,0.000126,0.001305,0.000350,0.000429,...,0.002305,0.000502,0.001635,0.000770,0.000344,0.000589,0.000313,0.000415,0.000464,0.000552,0.011179,0.016573,0.001073,0.000941,0.001532,0.000884,0.007527,0.000813,0.002352,0.000384,0.000427,0.002353,0.000225,0.000437,0.000959,0.001118,0.000214,0.000677,0.000680,0.000405,0.000147,0.000502,0.002468,0.008787,0.002552,0.000078,0.001163,0.001140,0.000269,0.001132
3,id_0027